In [1]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom

In [2]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/amattfel/Documents/GitHub'
proj_dir = '/mattfeld_2020/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [ ]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files['WHAT GOES HERE'])

In [68]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len('WHAT GOES HERE')))
#print('The timing of the TR was: {0} ms'.format(data_set.'WHAT GOES HERE'))
#print('The timing of the TE was: {0} ms'.format(???))
#print('The flip angle in degrees was: {0}'.format(???))
#print('The name of the scan was: {0}'.format(???))

# Why use the following format to access information in the header?
#print(data_set[int('00181312', 16)].value)
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value



The number of TRs collected equals: 66
The timing of the TR was: 6720 ms
The timing of the TE was: 80 ms
The flip angle in degrees was: 90
The name of the scan was: fMRI_DistortionMap_AP


In [ ]:
# How do I find phase encoding direction information?
import nibabel.nicom.csareader as csareader
csa_tr = csareader.read(csa_str)
csa_str = data_set[int('00291010', 16)].value
print(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0])
